# Referencias

In [1]:
# Referencia:
# https://www.kaggle.com/datasets/clmentbisaillon/fake-and-real-news-dataset/data

# Lib and data importing

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tqdm import tqdm
import tensorflow as tf
import numpy as np

In [3]:
# Loading Datasets
true = pd.read_csv('/content/drive/MyDrive/colab/True.csv')
fake = pd.read_csv('/content/drive/MyDrive/colab/Fake.csv')

# Data pre analising

In [4]:
true.shape , fake.shape

((21417, 4), (23481, 4))

In [5]:
len(true)/(len(true)+len(fake))

0.47701456635039424

In [6]:
# Verificando diferentes subjects
print('fake subjects= ', fake['subject'].unique())
print('true subjects= ', true['subject'].unique())
# Acredito que não fará diferença usar isto

fake subjects=  ['News' 'politics' 'Government News' 'left-news' 'US_News' 'Middle-east']
true subjects=  ['politicsNews' 'worldnews']


In [7]:
# Definindo o Target
fake = fake.assign(isfake=1)
true = true.assign(isfake=0)

# Juntando os dfs
df = pd.concat([fake, true], ignore_index=True)

In [8]:
# Para saber quantas datas são unicas.
df['date'].nunique()

2397

In [9]:
# Para saber quantos dados tem campos em branco.
df.isnull().sum()

title      0
text       0
subject    0
date       0
isfake     0
dtype: int64

In [10]:
# Removendo estas colunas, Se precisar rodo dnv com elas.
df = df.drop(['date', 'subject'], axis=1)

# Transformando Texto

###  Usando RoBERTa = tokenizer_roberta!

In [55]:
from transformers import RobertaTokenizer

# Carregar o tokenizer pré-treinado do modelo RoBERTa
tokenizer_roberta = RobertaTokenizer.from_pretrained("roberta-base")

# Exemplo de uso do tokenizer
text = "Esta é uma frase de exemplo."
tokens = tokenizer_roberta.tokenize(text)
print("Tokens RoBERTa:", tokens)

Tokens RoBERTa: ['Est', 'a', 'ĠÃ©', 'Ġu', 'ma', 'Ġfr', 'ase', 'Ġde', 'Ġexempl', 'o', '.']


In [12]:
# Exemplo de uso do tokenizer
text = "outro teste."
tokens = tokenizer_roberta.tokenize(text)
print("Tokens RoBERTa:", tokens)


Tokens RoBERTa: ['out', 'ro', 'Ġtest', 'e', '.']


### Usando GPT = tokenizer_gpt


In [13]:
from transformers import GPT2Tokenizer

# Carregar o tokenizer pré-treinado do modelo GPT
tokenizer_gpt = GPT2Tokenizer.from_pretrained("gpt2")

# Definir um token de padding
tokenizer_gpt.add_special_tokens({'pad_token': '[PAD]'})

# Exemplo de uso do tokenizer
text = "This is an example sentence."
tokens = tokenizer_gpt.tokenize(text)
print("Tokens GPT:", tokens)


Tokens GPT: ['This', 'Ġis', 'Ġan', 'Ġexample', 'Ġsentence', '.']


In [14]:
# Exemplo de uso do tokenizer
text = "outro teste."
tokens = tokenizer_gpt.tokenize(text)
print("Tokens GPT:", tokens)

Tokens GPT: ['out', 'ro', 'Ġtest', 'e', '.']


### Loading modelo

In [15]:
max_length_gpt=100

# Função para tokenizar texto usando o tokenizer
def tokenize_text(text, tokenizer, max_length_gpt):
    tokens = tokenizer(text, padding=True, truncation=True, max_length=max_length_gpt)
    input_ids = tokens['input_ids']
    attention_mask = tokens['attention_mask']
    return input_ids#, attention_mask

In [16]:
# Tamanho do vocabulário
vocab_size = len(tokenizer_gpt)
print("Tamanho do vocabulário:", vocab_size)

Tamanho do vocabulário: 50258


##### Explicaçãozinha:

Um tokenizador é uma parte fundamental do processamento de linguagem natural (NLP). Sua função principal é transformar um texto em uma sequência de tokens, que são unidades menores de linguagem, como palavras, subpalavras, caracteres ou até mesmo partes de palavras. Aqui está uma explicação passo a passo de como um tokenizador opera:

- **Divisão do texto em unidades menores:** O tokenizador recebe um texto como entrada e o divide em unidades menores, chamadas tokens. Esses tokens podem ser palavras individuais, partes de palavras, caracteres ou subpalavras, dependendo do tipo de tokenização usada e da linguagem específica.

- **Remoção de espaços em branco e pontuação:** Em muitos casos, o tokenizador remove espaços em branco adicionais e pontuações do texto para simplificar o processo de tokenização. Isso ajuda a garantir que os tokens resultantes sejam unidades significativas de linguagem.

- **Aplicação de regras de tokenização**: Dependendo da implementação específica do tokenizador e da língua, podem ser aplicadas várias regras de tokenização. Por exemplo, em inglês, as contrações como "can't" podem ser divididas em "can" e "'t", ou em algumas línguas, como o chinês, os tokens podem corresponder a caracteres individuais.

- **Tratamento de casos especiais:** Alguns tokenizadores lidam com casos especiais, como números, datas ou entidades nomeadas, de maneira específica. Por exemplo, números podem ser representados como um token único ou divididos em dígitos individuais.

- **Construção do vocabulário:** Durante o processo de tokenização, o tokenizador geralmente constrói um vocabulário que mapeia os tokens para índices numéricos. Isso é importante para representar o texto de entrada de uma maneira que um modelo de linguagem possa entender.

- **Saída dos tokens**: O tokenizador produz uma sequência de tokens que representa o texto de entrada. Esses tokens podem então ser usados como entrada para modelos de linguagem, como redes neurais, para realizar tarefas como classificação de texto, geração de texto, tradução automática, entre outras.

Em resumo, um tokenizador desempenha um papel fundamental no processamento de linguagem natural, transformando texto em uma representação numérica que os modelos de aprendizado de máquina podem entender e processar. Ele é uma etapa crucial em muitas aplicações de NLP.

<img src="https://miro.medium.com/v2/resize:fit:527/1*gWP5Whykah1101EpYy17qQ.png" alt="descrição da imagem">



1. **Input IDs (`input_ids`)**:
   - As 'input_ids' são uma sequência de números que representam cada token no texto após a tokenização.
   - Cada token é mapeado para um ID numérico específico de acordo com o vocabulário do modelo.
   - Esses IDs são o que o modelo de linguagem recebe como entrada durante o processo de treinamento ou inferência.
   - Por exemplo, a palavra "hello" pode ser mapeada para o ID 101 no vocabulário do modelo.

2. **Attention Mask (`attention_mask`)**:
   - A 'attention_mask' é uma sequência binária que indica ao modelo quais tokens são de fato tokens de entrada e quais são tokens de preenchimento (padding).
   - Os tokens de entrada recebem o valor 1, enquanto os tokens de preenchimento recebem o valor 0.
   - Essa máscara permite que o modelo saiba quais tokens devem ser considerados durante o cálculo da atenção.

Além desses, o tokenizer geralmente oferece outros atributos e métodos úteis, como:

- **Token Type IDs**: Usado em modelos que lidam com pares de sequências, como BERT. Indica a que sequência pertence cada token (sequência A ou sequência B).
- **Special Tokens**: Tokens especiais adicionados ao vocabulário do modelo, como tokens de início e fim de sequência, tokens de padding, tokens de separação de sequência, etc.
- **Métodos de Tokenização**: Métodos para tokenizar texto, como `encode()` para converter texto em IDs de tokens, `decode()` para converter IDs de tokens de volta para texto, entre outros.
- **Configurações e Parâmetros**: Opções para configurar o tokenizer, como tamanho máximo de sequência, adição de tokens especiais, etc.

Esses são alguns dos principais componentes e funcionalidades encontradas em um tokenizer usado em modelos de linguagem como o GPT-2. Cada modelo e biblioteca pode ter variações em seus atributos e métodos específicos.

#### Tokenizando

In [17]:
#Testando se a função funciona
df_separado = df.iloc[[0,2]].copy()

# Aplicar tqdm ao processar cada linha do DataFrame
tqdm.pandas()

# Aplicar a função de tokenização a cada linha do DataFrame
df_separado['text'] = df_separado['text'].progress_apply(lambda x: tokenize_text(x,tokenizer_roberta,max_length_gpt))

# Vendo se funcionou
df_separado

100%|██████████| 2/2 [00:00<00:00, 42.04it/s]


,title,text,isfake
0,Donald Trump Sends Out Embarrassing New Year’...,"[0, 19195, 140, 95, 1705, 326, 2813, 70, 1791,...",1
2,Sheriff David Clarke Becomes An Internet Joke...,"[0, 4148, 273, 6, 24, 21, 1487, 14, 320, 5819,...",1


In [18]:
a = df_separado.loc[0, 'text']
len(a)

100

In [19]:
# Atribuir os valores a uma variável
valores = df_separado.loc[0, 'text']

# Separar os valores individualmente
primeira_lista = valores

# Exibir os valores separados
print("Primeira lista:", primeira_lista)


Primeira lista: [0, 19195, 140, 95, 1705, 326, 2813, 70, 1791, 10, 9899, 188, 2041, 8, 989, 24, 23, 14, 4, 2978, 6, 37, 56, 7, 492, 10, 18066, 66, 7, 39, 11058, 6, 3988, 268, 8, 1437, 5, 182, 27820, 4486, 340, 433, 4, 1437, 20, 320, 2015, 311, 999, 56, 95, 65, 633, 7, 109, 8, 37, 1705, 326, 109, 24, 4, 287, 84, 5093, 6042, 11461, 3651, 8, 18369, 6, 38, 236, 7, 2813, 70, 9, 127, 964, 6, 2732, 6, 11058, 6, 3988, 268, 6, 8, 190, 5, 182, 27820, 24530, 491, 2454, 6, 10, 9899, 8, 2]


In [20]:
# Aplicar a função de tokenização a cada linha do DataFrame
df_token = df.copy()

# Aplicar tqdm ao processar cada linha do DataFrame
tqdm.pandas()

# Aplicar a função de tokenização a cada linha do DataFrame
df_token['text'] = df_token['text'].progress_apply(lambda x: tokenize_text(x,tokenizer_roberta,max_length_gpt))


100%|██████████| 44898/44898 [03:28<00:00, 215.51it/s]


### Transforming, Cleaning and Spliting Data

In [21]:
def preparar(df_escolhido):
    # Atribuir os valores a uma variável
    valores = df_escolhido['text']

    # Lista para armazenar os valores separados
    lista_separada = []

    # Loop através de cada linha do DataFrame
    for linha in valores:
        # Lista interna para armazenar os elementos individuais da linha
        lista_1 = []

        # Acessar a lista dentro da tupla
        lista_individual = linha

        # Adicionar os elementos de lista_individual a lista_1
        lista_1.extend(lista_individual)

        # Adicionar a lista interna à lista principal
        lista_separada.append(lista_1)

    # Converter lista_separada em DataFrame
    df_final = pd.DataFrame(lista_separada)

    return df_final

In [22]:
# Separando os dados em features (X) e alvo (y)
X = df_token[['text']]
y = df_token['isfake']

df_padded = preparar(X)

df_padded['isfake'] = y

In [23]:
df_padded.shape

(44898, 101)

In [24]:
df_padded['isfake'].value_counts()

isfake
1    23481
0    21417
Name: count, dtype: int64

In [25]:
df_padded = df_padded.dropna()

In [26]:
df_padded['isfake'].value_counts()

isfake
1    21263
0    19629
Name: count, dtype: int64

In [27]:
df_padded.isnull().sum()

0         0
1         0
2         0
3         0
4         0
         ..
96        0
97        0
98        0
99        0
isfake    0
Length: 101, dtype: int64

In [28]:
df_padded.shape

(40892, 101)

In [42]:
# Separando os dados em features (X) e alvo (y)
X = df_padded.drop(columns=['isfake'])
y = df_padded['isfake']

# Dividindo os dados em conjuntos de treinamento e teste (por exemplo, 80% treinamento, 20% teste)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Exibindo os tamanhos dos conjuntos de treinamento e teste
print("Tamanho do conjunto de treinamento:", len(X_train))
print("Tamanho do conjunto de teste:", len(X_test))
print("contagem de true e fake no teste:", y_train.value_counts())

Tamanho do conjunto de treinamento: 32713
Tamanho do conjunto de teste: 8179
contagem de true e fake no teste: isfake
1    17033
0    15680
Name: count, dtype: int64


### NeuralNetwork

In [30]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import random

In [31]:
# Convertendo DataFrame para numpy array
X_train_padded_np = X_train.values

# Converta X_train_padded para um tensor PyTorch com o mesmo tipo de dados dos parâmetros do modelo
X_train_padded_tensor = torch.tensor(X_train_padded_np, dtype=torch.float)

# Verifique o tipo de dados e o dispositivo do tensor
print("Tipo de dados:", X_train_padded_tensor.dtype)
print("Dispositivo:", X_train_padded_tensor.device)

Tipo de dados: torch.float32
Dispositivo: cpu


In [32]:
X_train_padded_tensor.shape

torch.Size([32713, 100])

In [47]:
class Model_lin(nn.Module):
    def __init__(self, entrada_dim, output_dim):
        super(Model_lin, self).__init__()
        self.camada_entrada = nn.Linear(entrada_dim, 50)
        self.camada_oculta = nn.Linear(50, 25)
        self.camada_saida = nn.Linear(25, output_dim)

    def forward(self, x):
        x = torch.relu(self.camada_entrada(x))
        x = torch.relu(self.camada_oculta(x))
        outputmodelo = torch.sigmoid(self.camada_saida(x))
        return outputmodelo

In [36]:
# Converta y_train para um tensor PyTorch
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float).view(-1, 1)

# Verifique novamente a forma da saída e dos rótulos verdadeiros
print("Forma de X_train_padded_tensor:", X_train_padded_tensor.shape)
print("Forma de y_train_tensor:", y_train_tensor.shape)

Forma de X_train_padded_tensor: torch.Size([32713, 100])
Forma de y_train_tensor: torch.Size([32713, 1])


In [37]:
X_train_padded_tensor.type(), y_train_tensor.type()

('torch.FloatTensor', 'torch.FloatTensor')

In [52]:
# Convertendo DataFrame para numpy array
X_test_padded_np = X_test.values

# Converta X_train_padded para um tensor PyTorch com o mesmo tipo de dados dos parâmetros do modelo
X_test_padded_tensor = torch.tensor(X_test_padded_np, dtype=torch.float)

# Converta y_train para um tensor PyTorch
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float).view(-1, 1)

In [51]:
# Inicialize o modelo
modelo = Model_lin(entrada_dim=max_length_gpt, output_dim=1)

# Defina a função de perda
lossfn = nn.BCELoss()

# Defina o otimizador
otimizador = optim.Adam(modelo.parameters(), lr=0.001)

# Número de épocas
num_epochs = 100

modelo.train()
# Loop de treinamento
for epoch in tqdm(range(num_epochs)):
    # Forward pass: Calcular a previsão (forward) do modelo
    output = modelo(X_train_padded_tensor)

    # Calcular a perda
    loss = lossfn(output, y_train_tensor)

    # Backward pass: Calcula o gradiente da perda em relação aos parâmetros do modelo
    otimizador.zero_grad()  # Zera os gradientes acumulados
    loss.backward()  # Calcula os gradientes
    otimizador.step()  # Atualiza os parâmetros

100%|██████████| 100/100 [00:09<00:00, 10.49it/s]


In [53]:
# Ative o modo de avaliação
modelo.eval()

# Faça previsões nos dados de teste
with torch.no_grad():
    outputs = modelo(X_test_padded_tensor)
    predicted_classes = (outputs > 0.5).float()  # Converta as previsões em classes binárias (0 ou 1)

# Calcule a precisão
accuracy = (predicted_classes == y_test_tensor).float().mean()
print(f'Acurácia: {accuracy.item()*100:.2f}%')

Acurácia: 51.72%


### RandomForest

In [41]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Inicialize o modelo Random Forest
modelo_rf = RandomForestClassifier(n_estimators=100, random_state=42)  # Você pode ajustar o número de árvores (n_estimators) conforme necessário

# Treine o modelo
modelo_rf.fit(X_train, y_train)

# Faça previsões nos dados de teste
previsoes_rf = modelo_rf.predict(X_test)

# Calcule a precisão
acuracia_rf = accuracy_score(y_test, previsoes_rf)
print(f'Acurácia do Random Forest: {acuracia_rf * 100:.2f}%')

Acurácia do Random Forest: 98.18%
